In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/eric/Downloads/[HackMTY2025]_ExpirationDateManagement_Dataset_v1.csv')

In [3]:
df

,Product_ID,Product_Name,Weight_or_Volume,LOT_Number,Expiry_Date,Quantity
0,COF006,Instant Coffee Stick,200ml,LOT-E19,2025-12-10,231
1,CHS010,Cheese Portion Pack,150g,LOT-E99,2025-10-29,484
2,SNK001,Snack Box Economy,150g,LOT-A68,2025-12-05,357
3,SNK001,Snack Box Economy,250ml,LOT-C79,2025-10-14,509
4,SNK001,Snack Box Economy,150g,LOT-B86,2025-12-12,500
...,...,...,...,...,...,...
145,JCE005,Orange Juice 200ml,200ml,LOT-D53,2025-12-22,495
146,JCE005,Orange Juice 200ml,180g,LOT-A73,2025-10-25,585
147,SNK001,Snack Box Economy,150g,LOT-E96,2025-10-08,253
148,SND004,Chicken Sandwich,150g,LOT-B17,2025-12-25,244


In [4]:
import requests

In [5]:
df["quantity"] = df["Weight_or_Volume"].str.extract(r"(\d+(?:\.\d+)?)").astype(float)
df["mlg"] = df["Weight_or_Volume"].str.extract(r"([a-zA-Z]+)")

In [6]:
df

,Product_ID,Product_Name,Weight_or_Volume,LOT_Number,Expiry_Date,Quantity,quantity,mlg
0,COF006,Instant Coffee Stick,200ml,LOT-E19,2025-12-10,231,200.0,ml
1,CHS010,Cheese Portion Pack,150g,LOT-E99,2025-10-29,484,150.0,g
2,SNK001,Snack Box Economy,150g,LOT-A68,2025-12-05,357,150.0,g
3,SNK001,Snack Box Economy,250ml,LOT-C79,2025-10-14,509,250.0,ml
4,SNK001,Snack Box Economy,150g,LOT-B86,2025-12-12,500,150.0,g
...,...,...,...,...,...,...,...,...
145,JCE005,Orange Juice 200ml,200ml,LOT-D53,2025-12-22,495,200.0,ml
146,JCE005,Orange Juice 200ml,180g,LOT-A73,2025-10-25,585,180.0,g
147,SNK001,Snack Box Economy,150g,LOT-E96,2025-10-08,253,150.0,g
148,SND004,Chicken Sandwich,150g,LOT-B17,2025-12-25,244,150.0,g


In [7]:
df["mlg"] = df["mlg"].replace({
    "ml": "Ml",
    "g": "g"
})

In [8]:
df["mlg"] = pd.Categorical(df["mlg"], categories=["Ml", "g"])

In [9]:
df

,Product_ID,Product_Name,Weight_or_Volume,LOT_Number,Expiry_Date,Quantity,quantity,mlg
0,COF006,Instant Coffee Stick,200ml,LOT-E19,2025-12-10,231,200.0,Ml
1,CHS010,Cheese Portion Pack,150g,LOT-E99,2025-10-29,484,150.0,g
2,SNK001,Snack Box Economy,150g,LOT-A68,2025-12-05,357,150.0,g
3,SNK001,Snack Box Economy,250ml,LOT-C79,2025-10-14,509,250.0,Ml
4,SNK001,Snack Box Economy,150g,LOT-B86,2025-12-12,500,150.0,g
...,...,...,...,...,...,...,...,...
145,JCE005,Orange Juice 200ml,200ml,LOT-D53,2025-12-22,495,200.0,Ml
146,JCE005,Orange Juice 200ml,180g,LOT-A73,2025-10-25,585,180.0,g
147,SNK001,Snack Box Economy,150g,LOT-E96,2025-10-08,253,150.0,g
148,SND004,Chicken Sandwich,150g,LOT-B17,2025-12-25,244,150.0,g


In [10]:
import qrcode
from PIL import Image

In [11]:
import pandas as pd
import uuid
import qrcode
import requests
import io

UPLOAD_URL = "http://localhost:8080/images/upload"

def generate_and_upload_qr(uuid_code):
    qr = qrcode.make(uuid_code)
    buffer = io.BytesIO()
    qr.save(buffer, format="PNG")
    buffer.seek(0)

    files = {"file": ("qr.png", buffer, "image/png")}
    response = requests.post(UPLOAD_URL, files=files)

    if response.status_code == 200:
        return response.text.strip()
    else:
        print(f"Error uploading QR for {uuid_code}: {response.text}")
        return None

df["uuid"] = [str(uuid.uuid4()) for _ in range(len(df))]

# Generar y subir los QR para cada fila
df["qr_url"] = df["uuid"].apply(generate_and_upload_qr)

df.to_csv("productos_con_qr.csv", index=False)

In [12]:
df = pd.read_csv('productos_con_qr.csv')

In [13]:
df.head()

,Product_ID,Product_Name,Weight_or_Volume,LOT_Number,Expiry_Date,Quantity,quantity,mlg,uuid,qr_url
0,COF006,Instant Coffee Stick,200ml,LOT-E19,2025-12-10,231,200.0,Ml,71d612c5-a14b-47ae-8217-cb89253d5ef4,https://firebasestorage.googleapis.com/v0/b/in...
1,CHS010,Cheese Portion Pack,150g,LOT-E99,2025-10-29,484,150.0,g,6633b8d2-5ddf-4143-b905-8351ac4d3992,https://firebasestorage.googleapis.com/v0/b/in...
2,SNK001,Snack Box Economy,150g,LOT-A68,2025-12-05,357,150.0,g,8bfed930-79d2-4dee-9640-4c6c00c6ef07,https://firebasestorage.googleapis.com/v0/b/in...
3,SNK001,Snack Box Economy,250ml,LOT-C79,2025-10-14,509,250.0,Ml,38a980f8-8919-4b3e-af4e-7ffa4bd47734,https://firebasestorage.googleapis.com/v0/b/in...
4,SNK001,Snack Box Economy,150g,LOT-B86,2025-12-12,500,150.0,g,4792acfc-2c8e-49cb-aad9-f8a67374ce9f,https://firebasestorage.googleapis.com/v0/b/in...


In [14]:
import requests
import pandas as pd
from datetime import datetime
import json

API_URL = "http://localhost:8080/productos/insertar"

def convert_df_row_to_json(row):
    expiry_date = pd.to_datetime(row['Expiry_Date'])
    if pd.isna(expiry_date.hour):
        expiry_date_str = expiry_date.strftime('%Y-%m-%dT00:00:00')
    else:
        expiry_date_str = expiry_date.strftime('%Y-%m-%dT%H:%M:%S')
    payload = {
        "uuidProduct": row['uuid'],
        "product_id": str(row['Product_ID']),
        "product_name": str(row['Product_Name']),
        "lotsName": str(row['LOT_Number']),
        "expiry_date": expiry_date_str,
        "quantity": int(row['Quantity']),
        "urlImage": str(row['qr_url']),
        "status": "VIGENTE",
        "mlg": str(row['mlg'])
    }
    
    return payload

def upload_productos(df):
    successful = 0
    failed = 0
    errors = []
    
    for index, row in df.iterrows():
        try:
            payload = convert_df_row_to_json(row)
            
            response = requests.post(
                API_URL,
                json=payload,
                headers={'Content-Type': 'application/json'}
            )
            
            if response.status_code == 200:
                successful += 1
                print(f"Producto {row['Product_ID']} subido exitosamente")
            else:
                failed += 1
                error_msg = f"Error en producto {row['Product_ID']}: {response.status_code} - {response.text}"
                print(error_msg)
                errors.append(error_msg)
                
        except Exception as e:
            failed += 1
            error_msg = f"Excepción en producto {row['Product_ID']}: {str(e)}"
            print(error_msg)
            errors.append(error_msg)
    
    print("\n" + "="*50)
    print(f"RESUMEN:")
    print(f"Exitosos: {successful}")
    print(f"Fallidos: {failed}")
    print(f"Total: {len(df)}")
    print("="*50)
    
    if errors:
        print("\nERRORES DETALLADOS:")
        for error in errors:
            print(error)
    
    return successful, failed, errors

print("Iniciando carga de productos...")
successful, failed, errors = upload_productos(df)

Iniciando carga de productos...
Producto COF006 subido exitosamente
Producto CHS010 subido exitosamente
Producto SNK001 subido exitosamente
Producto SNK001 subido exitosamente
Producto SNK001 subido exitosamente
Producto DRK002 subido exitosamente
Producto SNK001 subido exitosamente
Producto BIS007 subido exitosamente
Producto SAL008 subido exitosamente
Producto SND004 subido exitosamente
Producto MLK003 subido exitosamente
Producto BIS007 subido exitosamente
Producto FRU009 subido exitosamente
Producto FRU009 subido exitosamente
Producto SNK001 subido exitosamente
Producto FRU009 subido exitosamente
Producto COF006 subido exitosamente
Producto BIS007 subido exitosamente
Producto JCE005 subido exitosamente
Producto JCE005 subido exitosamente
Producto BIS007 subido exitosamente
Producto JCE005 subido exitosamente
Producto JCE005 subido exitosamente
Producto SNK001 subido exitosamente
Producto SNK001 subido exitosamente
Producto SAL008 subido exitosamente
Producto DRK002 subido exitosame